In [1]:
import pandas as pd

import numpy as np

/var/folders/6g/94mzvld16m75jj2nk4wk6jnr0000gp/T/ipykernel_2900/3461346634.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df=pd.read_csv('/Users/lindseygulden/dev/leg-up/projects/n2o/data/saha_n2o_data.csv')

In [3]:
df.head()

,Date,Year,Experiment,DataUse,Replication,Month,Vegetation,VegType,N2O,N_rate,...,PP7,AirT,DAF_TD,DAF_SD,WFPS25cm,NH4,NO3,Clay,Sand,SOM
0,2/9/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.896742,170,...,0.00,-2.0,276,241,0.666508,11.046340,22.940812,62.5,637.5,1.174072
1,2/10/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,2.190218,170,...,0.00,-2.4,277,242,0.640608,11.008087,22.959578,62.5,637.5,1.174072
2,2/18/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.542594,170,...,8.64,0.3,285,250,0.728085,10.831669,23.221928,62.5,637.5,1.174072
3,2/19/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.342870,170,...,8.13,-3.8,286,251,0.686872,10.849792,23.271978,62.5,637.5,1.174072
4,3/16/12,2012,BCSE_KBS,Building,R1,March,Corn,Annual,2.947778,170,...,8.39,17.6,312,277,0.716221,10.204748,24.206855,62.5,637.5,1.174072


In [4]:
df.describe()

,Year,N2O,N_rate,PP2,PP7,AirT,DAF_TD,DAF_SD,WFPS25cm,NH4,NO3,Clay,Sand,SOM
count,2246.000000,2246.000000,2246.000000,2246.000000,2246.000000,2246.000000,2246.000000,2246.000000,2194.000000,2170.000000,2216.000000,2246.000000,2246.000000,2246.000000
mean,2011.925200,7.468610,142.994212,5.219839,19.245273,12.408699,195.321015,198.898041,0.531047,11.451364,24.997063,134.425163,462.686739,1.901503
std,3.617418,28.488257,56.130412,10.064123,23.723416,9.490152,144.915448,142.907436,0.171868,13.214895,24.941785,79.564891,219.451957,1.132896
min,2002.000000,-7.415297,0.000000,0.000000,0.000000,-20.700000,1.000000,0.000000,0.024388,1.383393,0.370070,62.500000,55.000000,1.174072
25%,2011.000000,0.504316,135.000000,0.000000,3.560000,5.940000,73.250000,75.000000,0.390203,5.290323,10.184722,62.500000,418.833333,1.174072
50%,2013.000000,1.972058,170.000000,0.510000,11.930000,14.490000,160.000000,172.000000,0.553853,9.081282,19.795863,128.333333,491.666667,1.186800
75%,2014.000000,5.187618,170.000000,6.000000,27.000000,20.070000,308.750000,320.000000,0.680321,12.140325,27.085459,183.250000,637.500000,1.823200
max,2017.000000,593.072000,213.000000,95.250000,260.090000,30.700000,718.000000,678.000000,0.912113,228.525000,238.920000,280.000000,637.500000,4.500000


In [5]:
numeric_cols=df.select_dtypes([np.number]).columns
numeric_cols


Index(['Year', 'N2O', 'N_rate', 'PP2', 'PP7', 'AirT', 'DAF_TD', 'DAF_SD',
       'WFPS25cm', 'NH4', 'NO3', 'Clay', 'Sand', 'SOM'],
      dtype='object')

In [6]:
df[numeric_cols].corrwith(df['N2O'])

Year        0.005082
N2O         1.000000
N_rate      0.063759
PP2         0.151846
PP7         0.210514
AirT        0.121976
DAF_TD     -0.138786
DAF_SD     -0.162128
WFPS25cm    0.149632
NH4         0.497587
NO3         0.359227
Clay        0.147091
Sand       -0.173431
SOM         0.196683
dtype: float64

In [18]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import datetime as dt


In [46]:
def zero_pad(x):
    if len(str(x))==1:
        return '0'+str(x)
    else:
        return str(x)

def zero_pad_date(d,splitchar='/'):
    split_d=d.split(splitchar)
    split_d=[zero_pad(x) for x in split_d]
    return '/'.join(split_d)

In [61]:
df['date']=[dt.datetime.strptime(zero_pad_date(x),'%m/%d/%y') for x in df.Date]

In [62]:
df=pd.concat([df,pd.get_dummies(df.Vegetation,dtype='int')],axis=1)


In [66]:
numeric_columns=df.select_dtypes([np.number]).columns

In [67]:
X=df[[x for x in df[numeric_columns].columns.values if x != 'N2O']]
y=df['N2O']

In [68]:
x_train,x_test,y_train,y_test=train_test_split( X, y, test_size=0.33, random_state=42)

In [84]:
df[numeric_columns].corr()

,Year,N2O,N_rate,PP2,PP7,AirT,DAF_TD,DAF_SD,WFPS25cm,NH4,NO3,Clay,Sand,SOM,Corn,GLYMX,TRIAE
Year,1.000000,0.005082,0.507835,0.048260,0.034155,-0.174670,-0.217288,-0.159024,0.581966,0.071238,0.090134,-0.371359,0.238220,-0.093997,0.517232,-0.393165,-0.300842
N2O,0.005082,1.000000,0.063759,0.151846,0.210514,0.121976,-0.138786,-0.162128,0.149632,0.497587,0.359227,0.147091,-0.173431,0.196683,0.075588,-0.055396,-0.046059
N_rate,0.507835,0.063759,1.000000,0.045814,0.040572,-0.124720,-0.451276,-0.348424,0.472921,0.135401,0.251330,-0.236495,0.106406,0.020390,0.826666,-0.805386,-0.301002
PP2,0.048260,0.151846,0.045814,1.000000,0.520384,0.038463,-0.052178,-0.043300,0.167009,0.008547,0.030961,0.050715,-0.058429,0.061804,0.050699,-0.049155,-0.018703
PP7,0.034155,0.210514,0.040572,0.520384,1.000000,0.058929,-0.085922,-0.073653,0.184392,0.062815,-0.008435,0.040403,-0.035930,0.028725,0.006017,-0.052374,0.045059
AirT,-0.174670,0.121976,-0.124720,0.038463,0.058929,1.000000,-0.414874,-0.275817,-0.348544,0.154835,0.281925,0.077837,-0.051438,0.022922,-0.138222,0.087003,0.098747
DAF_TD,-0.217288,-0.138786,-0.451276,-0.052178,-0.085922,-0.414874,1.000000,0.636143,-0.072491,-0.249381,-0.348005,0.028405,0.041892,-0.107523,-0.420175,0.512814,0.047895
DAF_SD,-0.159024,-0.162128,-0.348424,-0.043300,-0.073653,-0.275817,0.636143,1.000000,0.005008,-0.163316,-0.290494,0.062592,-0.003061,-0.055650,-0.304653,0.418119,-0.012304
WFPS25cm,0.581966,0.149632,0.472921,0.167009,0.184392,-0.348544,-0.072491,0.005008,1.000000,0.109272,0.157472,-0.313674,0.165539,-0.005825,0.515922,-0.366607,-0.330492
NH4,0.071238,0.497587,0.135401,0.008547,0.062815,0.154835,-0.249381,-0.163316,0.109272,1.000000,0.535170,0.091247,-0.129575,0.156523,0.164417,-0.127579,-0.093806


In [87]:
list(df.columns.values)

['Date',
 'Year',
 'Experiment',
 'DataUse',
 'Replication',
 'Month',
 'Vegetation',
 'VegType',
 'N2O',
 'N_rate',
 'PP2',
 'PP7',
 'AirT',
 'DAF_TD',
 'DAF_SD',
 'WFPS25cm',
 'NH4',
 'NO3',
 'Clay',
 'Sand',
 'SOM',
 'date',
 'Corn',
 'GLYMX',
 'TRIAE',
 'prediction',
 'is_train']

In [69]:
regressor=XGBRegressor()

In [70]:
regressor.fit(x_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [72]:
y_predict=regressor.predict(x_test)

In [74]:
from sklearn.metrics import r2_score,mean_squared_error

In [76]:
print(f'R2 is {r2_score(y_test,y_predict)}')

R2 is 0.5362357446874223


In [77]:
print(f'RMSE is {mean_squared_error(y_test,y_predict)**0.5}')

RMSE is 18.755350560603347


In [78]:
df.N2O.std()

28.488257029488913

In [79]:
model_prediction=regressor.predict(X)

In [80]:
df['prediction']=model_prediction

In [82]:
df['is_train']=[1 if i in x_train.index else 0 for i in df.index]

In [83]:
df

,Date,Year,Experiment,DataUse,Replication,Month,Vegetation,VegType,N2O,N_rate,...,NO3,Clay,Sand,SOM,date,Corn,GLYMX,TRIAE,prediction,is_train
0,2/9/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.896742,170,...,22.940812,62.500000,637.500000,1.174072,2012-02-09,1,0,0,3.773072,1
1,2/10/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,2.190218,170,...,22.959578,62.500000,637.500000,1.174072,2012-02-10,1,0,0,2.665986,1
2,2/18/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.542594,170,...,23.221928,62.500000,637.500000,1.174072,2012-02-18,1,0,0,4.858080,0
3,2/19/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.342870,170,...,23.271978,62.500000,637.500000,1.174072,2012-02-19,1,0,0,4.201596,1
4,3/16/12,2012,BCSE_KBS,Building,R1,March,Corn,Annual,2.947778,170,...,24.206855,62.500000,637.500000,1.174072,2012-03-16,1,0,0,2.942730,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2241,10/16/13,2013,MCSE-T2,Testing,R4,October,TRIAE,Annual,1.640000,91,...,5.803681,128.333333,491.666667,1.806000,2013-10-16,0,0,1,1.845277,1
2242,11/15/13,2013,MCSE-T2,Testing,R1,November,TRIAE,Annual,0.990000,91,...,5.429873,162.500000,490.375000,1.496400,2013-11-15,0,0,1,0.738255,1
2243,11/15/13,2013,MCSE-T2,Testing,R2,November,TRIAE,Annual,-0.200000,91,...,7.239000,183.250000,432.875000,1.186800,2013-11-15,0,0,1,0.210211,1
2244,11/15/13,2013,MCSE-T2,Testing,R3,November,TRIAE,Annual,0.300000,91,...,8.434294,169.833333,418.833333,1.823200,2013-11-15,0,0,1,0.432097,0


In [ ]:
df[['Vegetation','prediction','N2O'.groupby('Vegetation').